In [100]:
import pandas as pd
import sklearn
from sklearn.preprocessing import QuantileTransformer, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import ExtraTreesClassifier
import xgboost
import numpy as np

## Preprocessing

In [90]:
df = pd.read_csv('data.csv')
df.drop(['Case_No'], inplace=True, axis=1)

In [98]:
qts = QuantileTransformer()
trX = qts.fit_transform(trX)
vaX = qts.transform(vaX)

def preprocessing(df):
    
    obj_cols = [x for x in df.columns if df[x].dtype == 'object']
    df_one_hot = pd.get_dummies(df, columns=obj_cols)
    trX, vaX = train_test_split(df_one_hot, test_size=.2, shuffle=641)
    df_tr = pd.DataFrame(trX, columns=df_one_hot.columns)
    df_va = pd.DataFrame(vaX, columns=df_one_hot.columns)
    tr_y = df_tr['Class/ASD Traits _No']
    va_y = df_va['Class/ASD Traits _No']
    df_tr = df_tr.iloc[:,:-2]
    df_va = df_va.iloc[:,:-2]
    
    return df_tr, df_va, tr_y, va_y

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/sklearn/preprocessing/_data.py:2357: UserWarning: n_quantiles (1000) is greater than the total number of samples (843). n_quantiles is set to n_samples.
  % (self.n_quantiles, n_samples))


In [99]:
df_tr, df_va, tr_y, va_y = preprocessing(df)

## Xgboost

In [95]:
param = {'max_depth': 2, 'eta': 1, 'objective': 'binary:logistic'}
param['eval_metric'] = ['auc']
num_round = 10
bst = xgboost.XGBRFClassifier(param=param)
clf_model = bst.fit(df_tr, tr_y)

[16:09:33] WARNING: ../src/learner.cc:516: 
Parameters: { param } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




In [96]:
score = clf_model.score(df_va, va_y)

## Ensemble

In [104]:
extra_trees = ExtraTreesClassifier(n_estimators=40, max_depth=10, max_features=25)
model = extra_trees.fit(df_tr, tr_y)

In [107]:
score = model.score(df_va, va_y)

In [109]:
import pickle
pkl_filename = "model.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(model, file)

In [112]:
with open(pkl_filename, 'rb') as file:
    load_model = pickle.load(file)